In [9]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import os

In [5]:
from nq_agents import multi_agent
from nq_agents import data_extraction

# Initialize the multi-agent system
my_agents = multi_agent.NQMultiAgent()

# Read input examples
GOLD_FILE = 'data/two-dev.jsonl'
num_examples = data_extraction.count_lines(GOLD_FILE)

input_examples = data_extraction.read_first_lines(GOLD_FILE, num_examples)

# Make predictions. side effect: write to file
save_path = GOLD_FILE.replace('.jsonl', '_predictions.jsonl')
predictions = my_agents.predict_batch(examples=input_examples, save_path=save_path, verbose=True)



Example 1/2
Question: can a ucc be filed on an individual
Initial answer: yes, a UCC - 1 financing statement can be filed on an individual
Cut answer: yes, a UCC - 1 financing statement can be filed on an individual
Refined answer: yes, a UCC can be filed on an individual
Refined answer: yes, a UCC - 1 can be filed on an individual
Refined answer: yes, a UCC - 1 financing statement can be filed on an individual
Refined answer: yes, a UCC - 1 can be filed on an individual
Refined answer: yes, a UCC - 1 can be filed on an individual
- - - - - - - - - - 
5
gold_answer: []
final_answer: yes, a UCC - 1 can be filed on an individual

Example 2/2
Question: who is the guy who walked across the twin towers
Initial answer: Philippe Petit
Cut answer: Petit
Refined answer: Philippe
Refined answer: Philippe
- - - - - - - - - - 
5
gold_answer: ['Philippe Petit', 'Philippe Petit', 'Philippe Petit', 'Philippe Petit', 'Philippe Petit']
final_answer: Philippe


### Evaluate


In [7]:
from nq_agents import evaluation

# Evaluate predictions
GOLD_FILE = 'data/two-dev.jsonl'
PREDICTIONS_FILE = 'data/two-dev_predictions.jsonl' # generated by multi_agent.py

result = evaluation.evaluate_predictions(GOLD_FILE, PREDICTIONS_FILE)

OrderedDict([('short-answer-n', 2), ('short-answer-f1', 0.6666666666666666), ('short-answer-precision', 1.0), ('short-answer-recall', 0.5)])


In [8]:
result

[(True, False, False, -1), (True, True, True, 50)]

In [9]:
a = {1:1, 2:2}
b = {1:1, 2:2}
a == b



True

In [10]:
a.keys().symmetric_difference(b.keys())

AttributeError: 'dict_keys' object has no attribute 'symmetric_difference'

In [ ]:
type(a.keys())